# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2024</br>

Name: Louise Ramos
</br>
Date: April 4, 2024
<br>
<br>
**At this time in the semester:** <br>
--We have explored a dataset. <br>
--We have cleaned our dataset. <br>
--We created a Github account with a repository for this class and included a Metadata file about our data. <br>
--We introduced general and more advanced SQL syntax, queries, and applications in R.<br>
--Created our own database from scratch in R using libraries RMariaDB and RSQLite <br>
--Populated our databases we created from scratch
--Created a visual enhanced entity relationship diagram for our database
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source.

We used two R packages to create two separate databases for practice in our previous assignment. Feel free to use your preferred package for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook/RMD/Visual Studio Code.<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. The cells below are the minimum requirement, but exploring larger queries are encouraged.<br>

**Data origin:**


In [ ]:
# Importing libraries
import pymysql
import pandas as pd
import sqlite3
import mysql.connector

In [ ]:
# Connecting to database
mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'Thissucks2020!',
    database = 'police_shootings'
)

In [6]:
# Creating a cursor 
cursor = mydb.cursor()

# SQL query to show table names
query = "SHOW TABLES"

# Running the query
cursor.execute(query)
tables = cursor.fetchall()

# Printing the table names
print("Tables")
for table in tables:
    print(table[0])

Tables
Incident
Location
Person


In [7]:
# Listing columns in each table
# Creating a cursor 
cursor = mydb.cursor()

# Listing table names
table_names = ['Incident', 'Location', 'Person']
for table_name in table_names:
    print(f"Columns in table '{table_name}':")

    # Retrieving column names for the current table
    query_columns = f"SHOW COLUMNS FROM {table_name}"
    cursor.execute(query_columns)
    columns = cursor.fetchall()

    # Printing the column names
    for column in columns:
        print(column[0])  

# Closing the cursor
cursor.close()

Columns in table 'Incident':
id
person_id
location_id
date
manner_of_death
armed
signs_of_mental_illness
threat_level
flee
body_camera
Columns in table 'Location':
id
city
state
longitude
latitude
is_geocoding_exact
Columns in table 'Person':
id
name
age
gender
race


True

In [8]:
# Testing to see if the tables are populated on the server
# Creating a cursor 
cursor = mydb.cursor()

# Listing the table names
table_names = ['Incident', 'Location', 'Person']
for table_name in table_names:
    print(f"\nTesting data in table '{table_name}':")

    # Query to fetch data from the tables
    query = f"SELECT * FROM {table_name} LIMIT 5"  
    cursor.execute(query)
    records = cursor.fetchall()

    # Displaying the data
    for record in records:
        print(record)

# Closing the cursor
cursor.close()


Testing data in table 'Incident':

Testing data in table 'Location':

Testing data in table 'Person':


True

#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Example Question:** What species of sea turtles were observed when the sea surface temperature was above 60 degrees (warm) and below 60 degrees (cold)?
<br>
**What tables are we joining?:** species and water_variables

In [ ]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.
#Remove the example question and insert yours.

**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** 
<br>
**What tables are we joining?:** 

In [ ]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.





**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** 
<br>
**What tables are we joining?:** 

In [ ]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.






**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Example Question:**  Which people are missing Age? (Limited to 5)
<br>

In [ ]:
#Replace the Example Question with your own and query the data for your question.
# Connecting to the database
conn = pymysql.connect(host='localhost', user='root', password='1734680nN', database='police_shootings')

# Creating a cursor object
cursor = conn.cursor()

# Executing SQL query to select rows with null age in the Person table
query = "SELECT * FROM Person WHERE age IS NULL LIMIT 5"

# Executing the query
cursor.execute(query)
rows_with_null_age = cursor.fetchall()

# Printing the results
print("Rows with NULL age:")
for row in rows_with_null_age:
    print(row)


**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Add the crew members of a future date to the database
<br>
**What table(s) are we adding a record to?** crew

In [ ]:
#Use INSERT INTO to manually insert data. 
insert = """
INSERT INTO Person (id, name, age, gender, race)
VALUES 
	(2000, "Josh", 29, M, W),
    (2001, "Lou", 24, F, H);"""



#It's okay if you do not execute your code if you do not want to add any data to your database.

**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statemts above. Find the minimum and maximum of a numerical column.
<br><br>
**Question:** Whats the youngest and oldest age in the data?
<br>
**What table(s) are we analyzing?** Person

In [ ]:
#Create a simple query to find the MIN() of a value and the MAX() of a value()
#They do not have to be in the same query yet.

# Creating a cursor object
cursor = conn.cursor()

# Querying to calculate MIN and MAX of age
query = """
    SELECT 
        MIN(CASE WHEN age IS NOT NULL THEN age ELSE NULL END) AS min_age,
        MAX(CASE WHEN age IS NOT NULL THEN age ELSE NULL END) AS max_age
    FROM 
        Person;
"""

# Executing the query
cursor.execute(query)
result = cursor.fetchone()

# Printing the result
print("Minimum Age (excluding NULL):", result[0])
print("Maximum Age (excluding NULL):", result[1])



Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** What were the names of the youngest person and oldest person who were shot in the states of New Jersey or Washington?
<br>

In [ ]:
#Combine the CASE statement with MIN() and MAX() for a more detailed query of your data.
#You can use one of the same CASE statements as above, or create a new one that relates to your MIN and MAX.

# Select the youngest person and oldest person murdered in either New Jersey or Washington
case_query = """SELECT  *
        FROM    police_shootings.Person p
        JOIN police_shootings.Locations l
            ON p.id = l.id
        SELECT 
                (CASE WHEN state IN ('WA', 'NJ') and MIN(age) THEN name END) as youngest
                (CASE WHEN state IN ('WA', 'NJ') and MAX (age) THEN name END) as oldest
"""

# Executing the query
cursor.execute(case_query)
people_results = cursor.fetchone()

# Printing the result
print("Youngest victim in either Washington or New Jersey:", result[0])
print("Oldest victim in either Washington or New Jersey:", result[1])


**AVG**<br>
AVG() will take the average of a numeric field.

**Question:** What is the average age of the people in the data?
<br>

In [ ]:
#Write a query to find the average of a numerical column.      

# Executing the SQL query 
query = """
    SELECT AVG(age) AS age
    FROM Person
    WHERE age IS NOT NULL;
"""

cursor.execute(query)
# Getting the results
result = cursor.fetchone()

# Printing the average age
print("Average age:", result[0])



Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above.

In [ ]:
#Add the AVG() aggregate function to your CASE query above that includes the MIN() and MAX()

# Getting the average age of people who died in Florida
avg_query = """SELECT  *
        FROM    police_shootings.Person p
        JOIN police_shootings.Locations l
            ON p.id = l.id
        SELECT 
                (CASE WHEN state=FL THEN AVG(age) END) as average_fl
"""

cursor.execute(avg_query)
# Getting the results
avg_result = cursor.fetchone()

# Printing the average age
print("Average age of victims in Florida:", result[0])



Now we are starting to get multiple values that we can save any time as a .csv if needed to access later.

In [ ]:
# Save your new result set as .csv file!





**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above. Be sure to obtain the same result set.

In [ ]:
#Rewrite your CASE/MIN/MAX/AVG code to include aliases if you have not done so already:

avg_query = """SELECT  *
        FROM    police_shootings.Person p
        JOIN police_shootings.Locations l
            ON p.id = l.id
        SELECT 
                (CASE WHEN state=FL THEN AVG(age) END) as average_fl
"""

cursor.execute(avg_query)
# Getting the results
avg_result = cursor.fetchone()

# Printing the average age
print("Average age of victims in Florida:", result[0])







**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Example Objective:** Delete all data from the Incident Table where manner_of_death is NULL.
<br>
**What table(s) are we deleting records from?** Incident

In [ ]:
#Replace the above info with your data. 
#You can write the code, but do not execute if you do not want to delete any of your data.

# DELETE FROM Incident WHERE manner_of_death IS NULL







We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>
<br>
We can also use dbExecute(con, DROP TABLE) from Jupyter Notebook/Rmarkdown/Visual Studio. You do not have to execute this code in this notebook if you do not want to delete a table.

In [ ]:
#Close the database connection :)
mydb.close()

**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. 